In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import dataframe_image as dfi
import matplotlib.pyplot as plt


In [2]:
"""
Loads in raw score file
Expecting the schema 
Name	Date	GF	GA	Team

Calcuates result, margies and goal difference
Does some prep work for Form and Win

Obviously names need to be consistant
"""
season = "S4_results"

raw_data = pd.read_csv("../data/"+season+".csv")
raw_data["GD"] = raw_data["GF"] - raw_data["GA"]
raw_data.loc[(raw_data['GD'] == 0), 'GamePoints'] = 0.5
raw_data.loc[(raw_data['GD'] < 0), 'GamePoints'] = -1
raw_data.loc[(raw_data['GD'] > 0), 'GamePoints'] = 1
raw_data.loc[(raw_data['GD'] > 6), 'GamePoints'] = 1.5
raw_data.loc[(raw_data['GD'] < -6), 'GamePoints'] = -1.5
raw_data.loc[(raw_data['GamePoints'] >= 1), 'W'] = 1
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'D'] = 1
raw_data.loc[(raw_data['GamePoints'] <= -1), 'L'] = 1
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'MW'] = 1
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'ML'] = 1

raw_data.loc[(raw_data['GamePoints'] >= 1), 'Result'] = '🔼'
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'Result'] = '➖'
raw_data.loc[(raw_data['GamePoints'] <= -1), 'Result'] = '🔽'
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'Result'] = '⏫'
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'Result'] = '⏬'

raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers

raw_data = raw_data.fillna(0)
pd.set_option('display.float_format', '{:.1f}'.format)
raw_data = raw_data.astype({'W':'int','D':'int','L':'int','MW':'int','ML':'int','GF':'int','GA':'int','GD':'int'})

raw_data['Date'] = pd.to_datetime(raw_data['Date'],dayfirst = True)
gameweek_int = raw_data.nunique().Date
gameweek = "Gameweek " + str(gameweek_int)

dates = pd.DataFrame(raw_data.sort_values("Date").Date.unique(),columns=['Date'])
dates['Gameweek'] = dates.index+1

raw_data = raw_data.merge(dates,how='right',on='Date')

pd.set_option('display.max_rows', 50)

display(raw_data)

<>:32: SyntaxWarning: invalid escape sequence '\*'
<>:32: SyntaxWarning: invalid escape sequence '\*'
/tmp/ipykernel_109222/522152173.py:32: SyntaxWarning: invalid escape sequence '\*'
  raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers


,Name,Date,GF,GA,Team,GD,GamePoints,W,D,L,MW,ML,Result,Gameweek
0,Calz,2025-05-22,13,10,Whites,3,1.0,1,0,0,0,0,🔼,1
1,Camilla,2025-05-22,13,10,Whites,3,1.0,1,0,0,0,0,🔼,1
2,Keysey,2025-05-22,13,10,Whites,3,1.0,1,0,0,0,0,🔼,1
3,Novak,2025-05-22,13,10,Whites,3,1.0,1,0,0,0,0,🔼,1
4,Venus Williams,2025-05-22,13,10,Whites,3,1.0,1,0,0,0,0,🔼,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,Snoop,2026-02-12,14,21,Colours,-7,-1.5,0,0,1,0,1,⏬,39
366,Gummy,2026-02-12,14,21,Colours,-7,-1.5,0,0,1,0,1,⏬,39
367,Gilet,2026-02-12,14,21,Colours,-7,-1.5,0,0,1,0,1,⏬,39
368,Reaper,2026-02-12,14,21,Colours,-7,-1.5,0,0,1,0,1,⏬,39


In [3]:
"""
Creates the Form Column
"""
def form_column(raw_data):
    raw_data['game_rank'] = raw_data.sort_values("Date").groupby("Name").Date.rank(ascending=False)
    raw_data_form = raw_data.loc[raw_data['game_rank'] <= 5]
    form = raw_data_form.groupby(['Name'], as_index=False).agg({'Result': ' '.join})
    return form

form = form_column(raw_data)

#display(form)

In [4]:
"""
Creates Matches Played (MP), Partipation (Par%), Pts and Win% (W%)

It also aggreates all the raw data to create an up to date table
"""
def get_totals(raw_data):
    played = raw_data.groupby('Name').count()
    played = pd.DataFrame(played["Date"]).rename(columns={"Date":"MP"})
    data = raw_data.groupby('Name').sum(numeric_only=True).sort_values(by=['GamePoints'],ascending=False).join(played).rename(columns={"GamePoints":"Pts"})

    data["Par%"] = data["MP"].div(gameweek_int)*100
    data["W%"] = data["W"]/data["MP"]*100
    
    data = data.astype({'Par%':'int','W%':'int'})
    data = data.sort_values(by=['Pts','GD'],ascending=False)
    return data

totals = get_totals(raw_data)



#display(totals)


In [6]:
"""
Join tables 
Drops unwanted columns 
Puts columns in the correct order
"""
table = totals.merge(form,on='Name').rename(columns={"Result":"Form"})
table.index = table.Name
table = table[['MP', 'W', 'D', 'L','MW', 'ML', 'GF', 'GA','GD','Par%','W%','Pts','Form']]
table.index.names = [gameweek]
#table.dfi.export('../outputs/table.png')
#table.dfi.export('../outputs/table.png', table_conversion='matplotlib')
table#.sort_values(by=['MP'],ascending=False)

,MP,W,D,L,MW,ML,GF,GA,GD,Par%,W%,Pts,Form
Gameweek 39,,,,,,,,,,,,,
JD Wetherspoon,22,13,2,7,5,0,345,281,64,56,59,9.5,🔼 🔽 ➖ 🔼 ⏫
Snoop,20,12,2,6,3,1,309,275,34,51,60,8.0,🔽 🔼 ➖ ⏫ ⏬
Gummy,28,17,1,10,2,3,428,421,7,71,60,7.0,➖ 🔼 ⏬ 🔽 ⏬
Schurrle,26,14,2,10,3,1,388,356,32,66,53,6.0,➖ 🔼 ⏫ 🔼 ⏫
Calz,16,9,2,5,2,1,222,204,18,41,56,5.5,🔽 🔼 ➖ 🔼 ⏫
Wilf,16,10,0,6,2,0,261,231,30,41,62,5.0,🔽 🔽 🔽 🔼 🔼
Camilla,15,7,1,7,2,1,213,198,15,38,46,1.0,🔼 ⏫ 🔼 🔼 ➖
Noel,19,10,0,9,2,2,283,279,4,48,52,1.0,🔽 🔼 🔼 🔽 🔽
Helmet,24,11,2,11,1,2,361,348,13,61,45,0.5,🔼 🔽 ➖ 🔽 ⏫


In [7]:
from datetime import datetime, timedelta

# Starting date
start_date = datetime.now() + timedelta(days=4)

# Number of times to repeat the process
iterations = 30

# Loop to run 30 times
for i in range(iterations):
    # Print the current date 10 times
    for j in range(10):
        print(start_date.strftime("%d/%m/%Y"))
    # Increment the date by 7 days
    start_date += timedelta(days=7)

28/01/2026
28/01/2026
28/01/2026
28/01/2026
28/01/2026
28/01/2026
28/01/2026
28/01/2026
28/01/2026
28/01/2026
04/02/2026
04/02/2026
04/02/2026
04/02/2026
04/02/2026
04/02/2026
04/02/2026
04/02/2026
04/02/2026
04/02/2026
11/02/2026
11/02/2026
11/02/2026
11/02/2026
11/02/2026
11/02/2026
11/02/2026
11/02/2026
11/02/2026
11/02/2026
18/02/2026
18/02/2026
18/02/2026
18/02/2026
18/02/2026
18/02/2026
18/02/2026
18/02/2026
18/02/2026
18/02/2026
25/02/2026
25/02/2026
25/02/2026
25/02/2026
25/02/2026
25/02/2026
25/02/2026
25/02/2026
25/02/2026
25/02/2026
04/03/2026
04/03/2026
04/03/2026
04/03/2026
04/03/2026
04/03/2026
04/03/2026
04/03/2026
04/03/2026
04/03/2026
11/03/2026
11/03/2026
11/03/2026
11/03/2026
11/03/2026
11/03/2026
11/03/2026
11/03/2026
11/03/2026
11/03/2026
18/03/2026
18/03/2026
18/03/2026
18/03/2026
18/03/2026
18/03/2026
18/03/2026
18/03/2026
18/03/2026
18/03/2026
25/03/2026
25/03/2026
25/03/2026
25/03/2026
25/03/2026
25/03/2026
25/03/2026
25/03/2026
25/03/2026
25/03/2026
01/04/2026

#### Who has played most together
#### Who has played the least together
#### Everybodys Most Played with Person
#### Best attacker Goals/Games Played
#### Best Defender Conceeded/Games Played
#### Most Wins in a Row
#### Most Losses in a Row
#### Most Margies
#### Most L_Margies
#### Which pair had the best winning rate
#### Which pair had the worst winning rate
#### Most games played in a row
#### Biggest table swing (after X games played)
#### GIF of table position over time
#### Which Team colour won more Darks vs Colours
#### Highest Scoring Game
#### Lowest Scoring Game
#### Ringer Stats
#### Most played Ringer
#### Most points Ringer
#### Whatsapp stats


In [ ]:
import pandas as pd
from IPython.display import Image
from IPython.display import Video

from itertools import combinations
from collections import defaultdict
# Set display options
pd.set_option('display.max_rows', None)        # Show all rows
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.width', None)           # Don't wrap to next line
pd.set_option('display.max_colwidth', None)    # Show full column contents

#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
#pd.reset_option('display.width')
#pd.reset_option('display.max_colwidth')


In [ ]:
Video("../outputs/Season3_cropped.mp4")

In [ ]:
Image("../data/bumpchat_visual.png")

In [ ]:
Image("../data/whatsapp_stats.png")

In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict
import numpy as np

def analyze_player_combinations(file_path, min_games_together=1):
    """
    Analyze player combinations from 5-a-side football data
    
    Args:
        file_path (str): Path to the CSV file containing game data
        min_games_together (int): Minimum number of games players must have played together to be included in stats
    
    Returns:
        dict: Analysis results including combination percentages
    """
    
    # Read the CSV file
    try:
        df = pd.read_csv(file_path)
    except:
        print("Error reading CSV file. Please check the file path.")
        return None
    
    # Clean column names (remove any extra spaces)
    df.columns = df.columns.str.strip()
    
    # Convert Date column to datetime if it's not already
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    
    # Remove any rows with missing essential data
    df = df.dropna(subset=['Name', 'Date', 'Team'])
    
    # Filter out ringers (players with asterisks in their names)
    df = df[~df['Name'].str.contains('\\*', na=False)]
    
    print(f"Analyzing data for {len(df)} player records across {df['Date'].nunique()} game dates")
    print(f"Players involved: {', '.join(sorted(df['Name'].unique()))}")
    print("-" * 60)
    
    # Group players by date and team to find teammates
    teammates_by_game = defaultdict(list)
    
    for date in df['Date'].unique():
        date_games = df[df['Date'] == date]
        
        for team in date_games['Team'].unique():
            team_players = date_games[date_games['Team'] == team]['Name'].tolist()
            if len(team_players) > 1:  # Only consider if there are multiple players
                teammates_by_game[(date, team)] = team_players
    
    # Count all possible player combinations and actual teammate combinations
    all_players = list(df['Name'].unique())
    total_possible_combinations = list(combinations(all_players, 2))
    
    # Count how many times each combination played together
    combination_counts = defaultdict(int)
    combination_games = defaultdict(list)
    
    for (date, team), players in teammates_by_game.items():
        if len(players) > 1:
            for combo in combinations(sorted(players), 2):
                combination_counts[combo] += 1
                combination_games[combo].append((date, team))
    
    # Count total games each player played
    player_game_counts = df['Name'].value_counts().to_dict()
    
    # Calculate combination percentages
    results = {
        'combination_stats': [],
        'player_stats': player_game_counts,
        'total_games': len(df['Date'].unique()),
        'total_combinations_found': len(combination_counts)
    }
    
    print(f"\nPLAYER COMBINATION ANALYSIS")
    print(f"{'Player 1':<12} {'Player 2':<12} {'Games Together':<15} {'% of P1 Games':<15} {'% of P2 Games':<15} {'Game Details'}")
    print("-" * 100)
    
    # Sort combinations by frequency and filter by minimum games
    sorted_combinations = sorted(combination_counts.items(), key=lambda x: x[1], reverse=True)
    sorted_combinations = [(combo, count) for combo, count in sorted_combinations if count >= min_games_together]
    
    if min_games_together > 1:
        print(f"\nNote: Only showing partnerships with {min_games_together}+ games together")
    
    for (player1, player2), count in sorted_combinations:
        p1_total_games = player_game_counts.get(player1, 0)
        p2_total_games = player_game_counts.get(player2, 0)
        
        p1_percentage = (count / p1_total_games * 100) if p1_total_games > 0 else 0
        p2_percentage = (count / p2_total_games * 100) if p2_total_games > 0 else 0
        
        # Get game details
        game_details = []
        for date, team in combination_games[(player1, player2)]:
            # Convert numpy datetime64 to pandas Timestamp for strftime
            date_str = pd.to_datetime(date).strftime('%d/%m/%Y')
            game_details.append(f"{date_str}({team})")
        
        game_details_str = ", ".join(game_details)
        
        print(f"{player1:<12} {player2:<12} {count:<15} {p1_percentage:<14.1f}% {p2_percentage:<14.1f}% {game_details_str}")
        
        results['combination_stats'].append({
            'player1': player1,
            'player2': player2,
            'games_together': count,
            'player1_percentage': round(p1_percentage, 1),
            'player2_percentage': round(p2_percentage, 1),
            'game_details': game_details
        })
    
    # Filter results for further analysis based on min_games_together
    filtered_combinations = [stat for stat in results['combination_stats'] if stat['games_together'] >= min_games_together]
    results['filtered_combination_stats'] = filtered_combinations
    
    # Summary statistics
    print(f"\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    print(f"Total players: {len(all_players)}")
    print(f"Total possible combinations: {len(total_possible_combinations)}")
    print(f"Combinations that actually played together: {len(combination_counts)}")
    print(f"Average games per player: {np.mean(list(player_game_counts.values())):.1f}")
    
    if combination_counts:
        combo_values = list(combination_counts.values())
        print(f"Most games together: {max(combo_values)}")
        print(f"Average games together (for combinations that played): {np.mean(combo_values):.1f}")
    
    print(f"\nIndividual player game counts:")
    for player, count in sorted(player_game_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {player}: {count} games")
    
    return results

def create_combination_matrix(results):
    """Create a matrix showing combination percentages"""
    if not results or not results['combination_stats']:
        return None
    
    # Get all unique players
    all_players = set()
    for stat in results['combination_stats']:
        all_players.add(stat['player1'])
        all_players.add(stat['player2'])
    
    all_players = sorted(list(all_players))
    
    # Create matrix
    matrix = pd.DataFrame(0.0, index=all_players, columns=all_players)
    
    for stat in results['combination_stats']:
        p1, p2 = stat['player1'], stat['player2']
        # Use average percentage for symmetric matrix
        avg_percentage = (stat['player1_percentage'] + stat['player2_percentage']) / 2
        matrix.loc[p1, p2] = avg_percentage
        matrix.loc[p2, p1] = avg_percentage
    
    return matrix

def analyze_best_worst_partners(results, min_games_together=1):
    """
    Analyze each player's most and least frequent partners
    
    Args:
        results (dict): Results from analyze_player_combinations
        min_games_together (int): Minimum number of games together to be considered
    """
    print(f"\n" + "="*80)
    print("BEST AND WORST PARTNERS BY FREQUENCY")
    print("="*80)
    print(f"{'Player':<15} {'Most Played With':<20} {'%':<8} {'Least Played With':<20} {'%':<8}")
    print("-" * 80)
    
    # Use filtered results
    combination_stats = results.get('filtered_combination_stats', results['combination_stats'])
    
    # Group combinations by player
    player_partnerships = defaultdict(list)
    
    for stat in combination_stats:
        p1, p2 = stat['player1'], stat['player2']
        p1_pct = stat['player1_percentage']
        p2_pct = stat['player2_percentage']
        games_together = stat['games_together']
        
        # Only include if meets minimum games requirement
        if games_together >= min_games_together:
            # Add partnership data for both players
            player_partnerships[p1].append((p2, p1_pct))
            player_partnerships[p2].append((p1, p2_pct))
    
    # Find best and worst for each player
    for player in sorted(player_partnerships.keys()):
        partnerships = player_partnerships[player]
        
        # Filter out 0% partnerships
        partnerships = [(partner, pct) for partner, pct in partnerships if pct > 0]
        
        if partnerships:
            # Sort by percentage
            partnerships.sort(key=lambda x: x[1], reverse=True)
            
            most_partner, most_pct = partnerships[0]
            least_partner, least_pct = partnerships[-1] if len(partnerships) > 1 else partnerships[0]
            
            print(f"{player:<15} {most_partner:<20} {most_pct:<7.1f}% {least_partner:<20} {least_pct:<7.1f}%")

def analyze_consecutive_weeks(file_path):
    """
    Rank players by most consecutive weeks played
    
    Args:
        file_path (str): Path to the CSV file
    
    Returns:
        dict: Player rankings by consecutive weeks
    """
    try:
        df = pd.read_csv(file_path)
    except:
        print("Error reading CSV file.")
        return None
    
    # Clean and filter data
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.dropna(subset=['Name', 'Date'])
    df = df[~df['Name'].str.contains('\\*', na=False)]  # Remove ringers
    
    # Get unique game dates sorted chronologically
    unique_dates = sorted(df['Date'].unique())
    
    print(f"\n" + "="*60)
    print("CONSECUTIVE WEEKS PLAYED RANKING")
    print("="*60)
    print(f"{'Rank':<6} {'Player':<15} {'Best Streak':<12} {'Current Streak':<15} {'Total Weeks':<12}")
    print("-" * 60)
    
    player_streaks = {}
    
    for player in df['Name'].unique():
        player_dates = set(df[df['Name'] == player]['Date'])
        
        # Track consecutive weeks
        current_streak = 0
        best_streak = 0
        current_streak_end = None
        
        for i, date in enumerate(unique_dates):
            if date in player_dates:
                current_streak += 1
                current_streak_end = date
                best_streak = max(best_streak, current_streak)
            else:
                current_streak = 0
        
        # Check if current streak is still active (ends with the last game date)
        is_current_active = current_streak_end == unique_dates[-1] if current_streak_end else False
        final_current_streak = current_streak if is_current_active else 0
        
        player_streaks[player] = {
            'best_streak': best_streak,
            'current_streak': final_current_streak,
            'total_weeks': len(player_dates)
        }
    
    # Sort by best streak, then by current streak, then by total weeks
    sorted_players = sorted(player_streaks.items(), 
                          key=lambda x: (x[1]['best_streak'], x[1]['current_streak'], x[1]['total_weeks']), 
                          reverse=True)
    
    for rank, (player, stats) in enumerate(sorted_players, 1):
        current_display = f"{stats['current_streak']} (active)" if stats['current_streak'] > 0 else "0"
        print(f"{rank:<6} {player:<15} {stats['best_streak']:<12} {current_display:<15} {stats['total_weeks']:<12}")
    
    return dict(sorted_players)

def analyze_table_movements(file_path, weeks_lookback=4):
    """
    Analyze biggest table movements over a specified number of weeks
    
    Args:
        file_path (str): Path to the CSV file
        weeks_lookback (int): Number of weeks to look back for movement analysis
    
    Returns:
        dict: Table movement analysis
    """
    try:
        df = pd.read_csv(file_path)
    except:
        print("Error reading CSV file.")
        return None
    
    # Clean and filter data
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.dropna(subset=['Name', 'Date', 'GF', 'GA'])
    df = df[~df['Name'].str.contains('\\*', na=False)]  # Remove ringers
    
    # Get unique dates sorted chronologically
    unique_dates = sorted(df['Date'].unique())
    
    if len(unique_dates) < weeks_lookback:
        print(f"Not enough data for {weeks_lookback} weeks analysis. Only {len(unique_dates)} weeks available.")
        weeks_lookback = len(unique_dates)
    
    print(f"\n" + "="*80)
    print(f"BIGGEST TABLE MOVEMENTS (LAST {weeks_lookback} WEEKS)")
    print("="*80)
    
    # Calculate league table for each week
    def calculate_table_for_dates(dates_range):
        filtered_df = df[df['Date'].isin(dates_range)]
        table = []
        
        for player in filtered_df['Name'].unique():
            player_games = filtered_df[filtered_df['Name'] == player]
            
            games_played = len(player_games)
            wins = len(player_games[player_games['GF'] > player_games['GA']])
            draws = len(player_games[player_games['GF'] == player_games['GA']])
            losses = games_played - wins - draws
            
            goals_for = player_games['GF'].sum()
            goals_against = player_games['GA'].sum()
            goal_difference = goals_for - goals_against
            
            points = (wins * 3) + draws
            
            table.append({
                'Player': player,
                'Games': games_played,
                'Points': points,
                'GD': goal_difference,
                'W': wins,
                'D': draws,
                'L': losses
            })
        
        # Sort by points, then goal difference, then goals for
        table.sort(key=lambda x: (x['Points'], x['GD'], goals_for), reverse=True)
        
        # Add positions
        for i, row in enumerate(table):
            row['Position'] = i + 1
        
        return table
    
    # Get current table (all weeks)
    current_table = calculate_table_for_dates(unique_dates)
    
    # Get table from N weeks ago
    weeks_ago_dates = unique_dates[:-weeks_lookback] if len(unique_dates) > weeks_lookback else []
    old_table = calculate_table_for_dates(weeks_ago_dates) if weeks_ago_dates else []
    
    # Calculate movements
    movements = []
    old_positions = {row['Player']: row['Position'] for row in old_table}
    
    for current_row in current_table:
        player = current_row['Player']
        current_pos = current_row['Position']
        old_pos = old_positions.get(player, None)
        
        if old_pos is not None:
            movement = old_pos - current_pos  # Positive = moved up, Negative = moved down
            movements.append({
                'Player': player,
                'Current_Pos': current_pos,
                'Old_Pos': old_pos,
                'Movement': movement,
                'Points': current_row['Points'],
                'GD': current_row['GD']
            })
        else:
            # New player (didn't play N weeks ago)
            movements.append({
                'Player': player,
                'Current_Pos': current_pos,
                'Old_Pos': 'New',
                'Movement': 'New',
                'Points': current_row['Points'],
                'GD': current_row['GD']
            })
    
    # Sort by biggest movements (up first, then down)
    movements.sort(key=lambda x: x['Movement'] if isinstance(x['Movement'], int) else -999, reverse=True)
    
    print(f"{'Player':<15} {'Current':<8} {'Previous':<9} {'Movement':<10} {'Points':<8} {'GD':<6}")
    print("-" * 70)
    
    for move in movements:
        old_pos_str = str(move['Old_Pos']) if move['Old_Pos'] != 'New' else 'New'
        
        if isinstance(move['Movement'], int):
            if move['Movement'] > 0:
                movement_str = f"↑{move['Movement']}"
            elif move['Movement'] < 0:
                movement_str = f"↓{abs(move['Movement'])}"
            else:
                movement_str = "="
        else:
            movement_str = "New"
        
        print(f"{move['Player']:<15} {move['Current_Pos']:<8} {old_pos_str:<9} {movement_str:<10} {move['Points']:<8} {move['GD']:<6}")
    
    return movements

def analyze_ringer_stats(file_path):
    """
    Analyze statistics for ringers (players with asterisks in their names)
    
    Args:
        file_path (str): Path to the CSV file
    
    Returns:
        dict: Ringer statistics
    """
    try:
        df = pd.read_csv(file_path)
    except:
        print("Error reading CSV file.")
        return None
    
    # Clean column names
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.dropna(subset=['Name', 'Date', 'GF', 'GA'])
    
    # Filter FOR ringers (players with asterisks in their names)
    ringers_df = df[df['Name'].str.contains('\\*', na=False)]
    
    if ringers_df.empty:
        print("No ringers found in the data.")
        return None
    
    print(f"\n" + "="*70)
    print("RINGER STATISTICS")
    print("="*70)
    print(f"{'Ringer Name':<20} {'Games':<8} {'Wins':<6} {'Draws':<6} {'Losses':<7} {'Win%':<7} {'GF':<5} {'GA':<5} {'GD':<6}")
    print("-" * 70)
    
    ringer_stats = []
    
    for ringer in sorted(ringers_df['Name'].unique()):
        ringer_games = ringers_df[ringers_df['Name'] == ringer]
        
        games_played = len(ringer_games)
        wins = len(ringer_games[ringer_games['GF'] > ringer_games['GA']])
        draws = len(ringer_games[ringer_games['GF'] == ringer_games['GA']])
        losses = games_played - wins - draws
        
        win_percentage = (wins / games_played * 100) if games_played > 0 else 0
        
        goals_for = ringer_games['GF'].sum()
        goals_against = ringer_games['GA'].sum()
        goal_difference = goals_for - goals_against
        
        ringer_stats.append({
            'Name': ringer,
            'Games': games_played,
            'Wins': wins,
            'Draws': draws,
            'Losses': losses,
            'Win_Percentage': win_percentage,
            'Goals_For': goals_for,
            'Goals_Against': goals_against,
            'Goal_Difference': goal_difference
        })
        
        print(f"{ringer:<20} {games_played:<8} {wins:<6} {draws:<6} {losses:<7} {win_percentage:<6.1f}% {goals_for:<5} {goals_against:<5} {goal_difference:<6}")
    
    # Summary statistics
    if ringer_stats:
        total_games = sum(stat['Games'] for stat in ringer_stats)
        total_wins = sum(stat['Wins'] for stat in ringer_stats)
        overall_win_rate = (total_wins / total_games * 100) if total_games > 0 else 0
        
        print("-" * 70)
        print(f"{'TOTALS':<20} {total_games:<8} {total_wins:<6} {sum(stat['Draws'] for stat in ringer_stats):<6} {sum(stat['Losses'] for stat in ringer_stats):<7} {overall_win_rate:<6.1f}% {sum(stat['Goals_For'] for stat in ringer_stats):<5} {sum(stat['Goals_Against'] for stat in ringer_stats):<5} {sum(stat['Goal_Difference'] for stat in ringer_stats):<6}")
    
    return ringer_stats

def analyze_win_loss_streaks(file_path):
    """
    Analyze win/loss streaks for each player including margies
    
    Args:
        file_path (str): Path to the CSV file
    """
    try:
        df = pd.read_csv(file_path)
    except:
        print("Error reading CSV file.")
        return None
    
    # Clean and filter data
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.dropna(subset=['Name', 'Date', 'Team', 'GF', 'GA'])
    df = df[~df['Name'].str.contains('\\*', na=False)]  # Remove ringers
    
    # Sort by date to analyze streaks chronologically
    df = df.sort_values(['Date', 'Name'])
    
    print(f"\n" + "="*60)
    print("WIN/LOSS STREAKS ANALYSIS")
    print("="*60)
    print(f"{'Player':<15} {'Best Win Streak':<18} {'Worst Loss Streak':<18}")
    print("-" * 60)
    
    player_streaks = {}
    
    for player in sorted(df['Name'].unique()):
        player_games = df[df['Name'] == player].sort_values('Date')
        
        # Determine win/loss for each game
        results = []
        for _, game in player_games.iterrows():
            gf, ga = game['GF'], game['GA']
            if gf > ga:
                if gf - ga >= 6:  # Margie win
                    results.append('W(M)')
                else:
                    results.append('W')
            elif ga > gf:
                if ga - gf >= 6:  # Margie loss
                    results.append('L(M)')
                else:
                    results.append('L')
            else:
                results.append('D')  # Draw
        
        # Calculate streaks
        if results:
            win_streaks = []
            loss_streaks = []
            current_win_streak = 0
            current_loss_streak = 0
            
            for result in results:
                if result.startswith('W'):
                    current_win_streak += 1
                    if current_loss_streak > 0:
                        loss_streaks.append(current_loss_streak)
                        current_loss_streak = 0
                elif result.startswith('L'):
                    current_loss_streak += 1
                    if current_win_streak > 0:
                        win_streaks.append(current_win_streak)
                        current_win_streak = 0
                else:  # Draw
                    if current_win_streak > 0:
                        win_streaks.append(current_win_streak)
                        current_win_streak = 0
                    if current_loss_streak > 0:
                        loss_streaks.append(current_loss_streak)
                        current_loss_streak = 0
            
            # Don't forget the final streak
            if current_win_streak > 0:
                win_streaks.append(current_win_streak)
            if current_loss_streak > 0:
                loss_streaks.append(current_loss_streak)
            
            best_win_streak = max(win_streaks) if win_streaks else 0
            worst_loss_streak = max(loss_streaks) if loss_streaks else 0
            
            player_streaks[player] = {
                'best_win_streak': best_win_streak,
                'worst_loss_streak': worst_loss_streak
            }
            
            print(f"{player:<15} {best_win_streak:<18} {worst_loss_streak:<18}")
    
    return player_streaks

def analyze_partnership_records(file_path, min_games_together=1):
    """
    Analyze win/loss records for player partnerships
    
    Args:
        file_path (str): Path to the CSV file
        min_games_together (int): Minimum number of games together to be considered
    """
    try:
        df = pd.read_csv(file_path)
    except:
        print("Error reading CSV file.")
        return None
    
    # Clean and filter data
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.dropna(subset=['Name', 'Date', 'Team', 'GF', 'GA'])
    df = df[~df['Name'].str.contains('\\*', na=False)]  # Remove ringers
    
    # Group by date and team to find partnerships and their results
    partnership_records = defaultdict(lambda: {'wins': 0, 'losses': 0, 'draws': 0})
    
    for date in df['Date'].unique():
        date_games = df[df['Date'] == date]
        
        for team in date_games['Team'].unique():
            team_data = date_games[date_games['Team'] == team]
            team_players = team_data['Name'].tolist()
            
            if len(team_players) > 1:
                # Get team result (assuming all players on same team have same GF/GA)
                gf = team_data['GF'].iloc[0]
                ga = team_data['GA'].iloc[0]
                
                # Determine result
                if gf > ga:
                    result_key = 'wins'
                elif ga > gf:
                    result_key = 'losses'
                else:
                    result_key = 'draws'
                
                # Record result for all partnerships in this team
                for combo in combinations(sorted(team_players), 2):
                    partnership_records[combo][result_key] += 1
    
    # Calculate win rates and find best/worst partnerships for each player
    print(f"\n" + "="*90)
    print("BEST AND WORST PARTNERSHIP RECORDS")
    if min_games_together > 1:
        print(f"(Only showing partnerships with {min_games_together}+ games together)")
    print("="*90)
    print(f"{'Player':<12} {'Best Partner':<15} {'W-L-D':<10} {'Win%':<8} {'Worst Partner':<15} {'W-L-D':<10} {'Win%':<8}")
    print("-" * 90)
    
    player_partnership_records = defaultdict(list)
    
    # Group partnerships by individual players - only include those meeting min_games requirement
    for (p1, p2), record in partnership_records.items():
        total_games = record['wins'] + record['losses'] + record['draws']
        if total_games >= min_games_together:  # Apply the filter here
            win_rate = (record['wins'] / total_games) * 100
            record_str = f"{record['wins']}-{record['losses']}-{record['draws']}"
            
            player_partnership_records[p1].append((p2, win_rate, record_str, total_games))
            player_partnership_records[p2].append((p1, win_rate, record_str, total_games))
    
    # Find best and worst for each player
    for player in sorted(player_partnership_records.keys()):
        partnerships = player_partnership_records[player]
        
        if partnerships:
            # Sort by win rate, then by total games for tie-breaking
            partnerships.sort(key=lambda x: (x[1], x[3]), reverse=True)
            
            best_partner, best_rate, best_record, _ = partnerships[0]
            worst_partner, worst_rate, worst_record, _ = partnerships[-1] if len(partnerships) > 1 else partnerships[0]
            
            print(f"{player:<12} {best_partner:<15} {best_record:<10} {best_rate:<7.1f}% {worst_partner:<15} {worst_record:<10} {worst_rate:<7.1f}%")

# Example usage
if __name__ == "__main__":
    # Replace with your actual CSV file path
    file_path = "../data/S3_results.csv"
    
    # Set minimum games together (default is 1, but you can change this)
    min_games = 1  # Change this value to filter partnerships
    
    print("5-A-SIDE FOOTBALL PLAYER COMBINATION ANALYZER")
    print("=" * 50)
    
    # Analyze the data
    results = analyze_player_combinations(file_path, min_games_together=min_games)
    
    if results:
        # NEW: Analyze consecutive weeks played
        analyze_consecutive_weeks(file_path)
        
        # NEW: Analyze table movements (change the number to adjust weeks lookback)
        analyze_table_movements(file_path, weeks_lookback=4)
        
        # NEW: Analyze ringer statistics
        analyze_ringer_stats(file_path)
        
        # Analyze best and worst partners
        analyze_best_worst_partners(results, min_games_together=min_games)
        
        # Analyze win/loss streaks
        analyze_win_loss_streaks(file_path)
        
        # Analyze partnership win/loss records
        analyze_partnership_records(file_path, min_games_together=min_games)
        
        print(f"\n" + "="*60)
        print("CREATING COMBINATION MATRIX")
        print("="*60)
        
        # Create and display combination matrix
        matrix = create_combination_matrix(results)
        if matrix is not None:
            print("\nCombination Matrix (Average % of games played together):")
            print(matrix.round(1))
            
            # Save matrix to Excel if needed
            # matrix.to_excel("player_combination_matrix.xlsx")
            # print("\nMatrix saved to 'player_combination_matrix.xlsx'")
    
    print("\nAnalysis complete!")